# Setup


In [44]:
%cd ..
!eza --icons

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2
 data   images   README.md   src


In [2]:
!dvc init -f

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
!git status
!git commit -m "🎉 init: initialised dvc"

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md
	data/
	images/
	src/



[main (root-commit) abf5167] 🎉 init: initialised dvc
 3 files changed, 6 insertions(+)
 create mode 100755 .dvc/.gitignore
 create mode 100755 .dvc/config
 create mode 100755 .dvcignore


## Decoupling storage and compute

I added google drive as remote storage for dvc.


In [5]:
!dvc remote add -d remotestorage gdrive://1sK3RLupgUrr1YKd3A8BOjpwg5AAVqV3p

Setting 'remotestorage' as a default remote.


In [7]:
!git add .dvc/config
!git status
!git commit -m "⚡ update: updated dvc config"

On branch main
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .dvc/config

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md
	data/
	images/
	src/

[main 0bd0ae0] ⚡ update: updated dvc config
 1 file changed, 4 insertions(+)


In [45]:
%cd src/

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/src


# Imports


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
import warnings

warnings.simplefilter("ignore")

# Loading the dataset and splitting it


I use the following code to clean the `emails.csv` and save it as `raw_data.csv` . This code is from my assignment 1 [ `prepare.ipynb` ](https://github.com/Roudranil/applied-ml-assignment-1/blob/main/src/prepare.ipynb)

```python
def startstrip(text: str, subtext: str):
    if text.startswith(subtext):
        text = text[len(subtext) :]
    return text

def clean_text(text: str):
    # convert text to lower case to make it easier to do the preprocessing
    text = text.lower()

    # remove the "subject :" from the beginning
    text = startstrip(text, "subject: ")
    text = text.lstrip("subject: ")

    # finding if the mail was a reply to another mail
    is_reply = 1 if text.startswith("re :") else 0
    text = startstrip(text, "re : ")

    # cleaning the text
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"[^\x00-\x7F]+", "", text)

    stopwords_list = stopwords.words("english")
    text = " ".join(word for word in text.split() if word not in stopwords_list)

    return text, is_reply

cleaned_data = pd.DataFrame(
    data["text"].apply(lambda x: clean_text(x)).tolist(), columns=["text", "is_reply"]
)
cleaned_data["spam"] = data["spam"]

cleaned_data.to_csv("../data/raw_data.csv", index=False)
```


In [13]:
preprocessed_raw_data = pd.read_csv("../data/raw_data.csv")
# Split the data into train/validation/test with the specified random seeds
train, test = train_test_split(preprocessed_raw_data, test_size=0.2, random_state=42)
train, val = train_test_split(
    train, test_size=0.25, random_state=42
)  # 0.25 x 0.8 = 0.2

In [15]:
# Save the splits in the 'data' folder
train.to_csv("../data/train.csv", index=False)
val.to_csv("../data/validation.csv", index=False)
test.to_csv("../data/test.csv", index=False)

### Tracking dataset versions with dvc


In [46]:
%cd ../data
!eza --icons

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/data
 raw_data.csv   test.csv   train.csv   validation.csv


In [47]:
!dvc add raw_data.csv test.csv train.csv validation.csv

⠋ Checking graph                                       core>
  0% Adding...|                       | raw_data.csv |0/4 [00:00<?,     ?file/s]
!
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Adding raw_data.csv to cache          0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /data/cmi/notes/sem-4/app0/1 [00:00<?,    ?files/s]
 25% Adding...|█████▊                 | test.csv |1/4 [00:00<00:00,  7.31file/s]
!
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Adding test.csv to

In [53]:
!git status
!git add train.csv.dvc validation.csv.dvc test.csv.dvc raw_data.csv.dvc .gitignore
!git commit -m "⚡ update: enabled versioning data with dvc"

On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../README.md
	./
	../images/
	../src/

nothing added to commit but untracked files present (use "git add" to track)


[main 182e994] ⚡ update: enabled versioning data with dvc
 5 files changed, 28 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/raw_data.csv.dvc
 create mode 100644 data/test.csv.dvc
 create mode 100644 data/train.csv.dvc
 create mode 100644 data/validation.csv.dvc
